In [83]:
import sys
sys.path.append("..")
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from dory.scaler.standard import StandardScaler
from dory.linear.ols import OrdinaryLeastSquares
from sklearn.linear_model import LinearRegression

In [84]:
artificial_df = pd.read_csv("artificial1d.csv", header=None)
artificial_df.describe()

,0,1
count,3.000000e+01,30.000000
mean,-2.960595e-17,0.067618
std,6.071316e-01,1.065356
min,-1.000000e+00,-2.082017
25%,-5.000000e-01,-0.870920
50%,0.000000e+00,0.098189
75%,5.000000e-01,0.957162
max,1.000000e+00,2.309838


In [85]:
X = artificial_df.iloc[:, :-1]
y = artificial_df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42069)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(24, 1) (6, 1) (24,) (6,)


In [ ]:
ols = OrdinaryLeastSquares()
model = LinearRegression(fit_intercept=False)
ols.fit(X_train, y_train)
model.fit(X_train, y_train)
parameters = ols.get_parameters()
print(f"Coefficients: {parameters}")
print(f"Coefficients: {model.intercept_}")

Coefficients: 0    1.558998
dtype: float64
Coefficients: [1.55899847]
